This notebook downloads all the stock data needed for the alpha analysis, computes the alpha for given stocks, and creates a dataset with necessary information for further analysis

In [1]:
import yfinance as yf
from fredapi import Fred
import numpy as np
import pandas as pd
from scipy.stats import linregress
from tqdm import tqdm
from time import sleep 
import pickle
import os

In [2]:
# this contains the companies in S&P500
# you can download from this link: https://github.com/datasets/s-and-p-500-companies/blob/main/data/constituents.csv
tickers = pd.read_csv('constituents.csv')

In [3]:
symbols = list(tickers['Symbol'])

# adding the symbol for the index so we can download data for it as well
symbols.append('^GSPC') 
print(len(symbols))

503


In [4]:
# the start and end of the analysis period
start = '2009-01-01'
end = '2024-12-19'

In [5]:
# downloading historical financial data from yahoo finance 
yf_data_file = "yf_data.pkl"
# no need to redownload the data if it already exists 
if os.path.exists(yf_data_file):
    with open("yf_data.pkl", "rb") as file:
        yf_data = pickle.load(file)
else:
    yf_data = {}
    for symbol in symbols:
        symbol_data = yf.download(symbol, start=start, end=end)
        yf_data[symbol] = symbol_data
        sleep(5)

    # update dates 
    for stock, data_df in yf_data.items():
        data_df = data_df.reset_index()
        data_df["New_Date"] = pd.to_datetime(data_df["Date"])
        yf_data[stock] = data_df

    print(len(yf_data))
    
    with open(yf_data_file, "wb") as file:
        pickle.dump(yf_data, file)
    

In [6]:
# this is the function which reutrns the stock price data for a specific time period
def get_stock_data_for_period(ticker, start_date, end_date):
    df = yf_data[ticker].reset_index()

    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    filtered_df = df[(df["New_Date"] >= start_date) & (df["New_Date"] <= end_date)]
    filtered_df.columns = filtered_df.columns.droplevel(1)
    return filtered_df


In [7]:
# get the FRED API key from the endironment
fred_api_key = os.getenv("FRED_API_KEY")
if not fred_api_key:
    print("Need to provide a FRED API key in the FRED_API_KEY env variable...")

In [8]:
fred = Fred(api_key=fred_api_key)

In [9]:
test_df = get_stock_data_for_period('BLDR', '2009-01-01', '2009-06-01')
test_df

Price,index,Date,Adj Close,Close,High,Low,Open,Volume,New_Date
0,0,2009-01-02,1.51,1.51,1.56,1.49,1.56,44100,2009-01-02
1,1,2009-01-05,1.52,1.52,1.56,1.47,1.56,243500,2009-01-05
2,2,2009-01-06,1.35,1.35,1.54,1.35,1.50,86100,2009-01-06
3,3,2009-01-07,1.27,1.27,1.38,1.27,1.37,74000,2009-01-07
4,4,2009-01-08,1.35,1.35,1.38,1.34,1.35,269000,2009-01-08
...,...,...,...,...,...,...,...,...,...
98,98,2009-05-26,3.52,3.52,3.55,3.01,3.12,71800,2009-05-26
99,99,2009-05-27,3.72,3.72,3.89,3.59,3.59,63100,2009-05-27
100,100,2009-05-28,4.01,4.01,4.04,3.69,3.75,91500,2009-05-28
101,101,2009-05-29,4.11,4.11,4.20,3.91,4.13,101200,2009-05-29


In [10]:
# TB3MS corresponds to the "3-Month Treasury Bill Secondary Market Rate"
# described in https://fred.stlouisfed.org/series/TB3MS

tb3ms = fred.get_series('TB3MS', observation_start=start, observation_end=end)
print(len(tb3ms))
print(tb3ms.mean())

192
1.14265625


In [12]:
# Creating a dataframe with the FRED data
fred_df = tb3ms.reset_index().rename(columns={"index": "Date", 0: "Value"})
fred_df["New_Date"] = pd.to_datetime(fred_df["Date"])
fred_df

,Date,Value,New_Date
0,2009-01-01,0.13,2009-01-01
1,2009-02-01,0.30,2009-02-01
2,2009-03-01,0.21,2009-03-01
3,2009-04-01,0.16,2009-04-01
4,2009-05-01,0.18,2009-05-01
...,...,...,...
187,2024-08-01,5.05,2024-08-01
188,2024-09-01,4.72,2024-09-01
189,2024-10-01,4.51,2024-10-01
190,2024-11-01,4.42,2024-11-01


In [13]:
# this is the function which returns the FRED data (TB3MS) for a specific time period 
def get_fred_data_for_period(start, end):
    start_date = pd.to_datetime(start)
    end_date = pd.to_datetime(end)
    
    fred_filtered_df = fred_df[(fred_df["New_Date"] >= start_date) & (fred_df["New_Date"] <= end_date)]
    return fred_filtered_df


In [14]:
test = get_fred_data_for_period('2023-01-01', '2023-06-01')
test

,Date,Value,New_Date
168,2023-01-01,4.54,2023-01-01
169,2023-02-01,4.65,2023-02-01
170,2023-03-01,4.69,2023-03-01
171,2023-04-01,4.92,2023-04-01
172,2023-05-01,5.14,2023-05-01
173,2023-06-01,5.16,2023-06-01


In [15]:
# function to compute the alpha of a specific stock and period of time
# stock_ticker: stock to compute alpha for 
# start_date: staring date for the period
# end_date: ending date for the period
# 
# Returns
# - true if we had enough data to compute alpha  
# - alpha 
# - beta
def alphabeta(stock_ticker, start_date, end_date):
    # get the stock price data for the interval
    stock_data = get_stock_data_for_period(stock_ticker, start_date=start_date, end_date=end_date)

    # get the index price data for the interval 
    market_ticker = "^GSPC"
    market_data = get_stock_data_for_period(market_ticker, start_date=start_date, end_date=end_date)

    # return here if we do not have stock data for this period, the stock may 
    # have not existed back then
    if stock_data.empty:
        return False, 0, 0

    # compute percent change of the stock price for the time period 
    stock_beggining_value = stock_data.head(1)['Adj Close'].values[0]
    stock_end_value = stock_data.tail(1)['Adj Close'].values[0]
    stock_overall_change = ((stock_end_value - stock_beggining_value)/stock_beggining_value)*100

    # compute percent change of the index price for the time period
    market_beggining_value = market_data.head(1)['Adj Close'].values[0]
    market_end_value = market_data.tail(1)['Adj Close'].values[0]
    market_overall_change = ((market_end_value - market_beggining_value)/market_beggining_value)*100

    # compute the beta as the intercept of the linear regression of the.. 
    stock_data['close_diff_stock'] = stock_data['Adj Close'].pct_change().dropna()
    market_data['close_diff_market'] = market_data['Adj Close'].pct_change().dropna()
    aligned_data = pd.merge(stock_data, market_data, on = "Date", how="inner").dropna() # sort_index().dropna()
    beta, intercept, r_value, p_value, std_err = linregress(aligned_data['close_diff_market'], aligned_data['close_diff_stock'])

    # compute the avg risk free rate for the period 
    tb3ms = get_fred_data_for_period(start=start_date, end=end_date)
    risk_free_rate = tb3ms['Value'].mean()
    
    # compute alpha
    alpha = stock_overall_change - (risk_free_rate + (beta*(market_overall_change - risk_free_rate)))
    return True, alpha/100,beta

In [16]:
# checking 
worked, alpha,beta = alphabeta('PLTR', '2024-06-01','2024-12-24')
print(alpha,beta)

2.1638634339603304 2.237353181107114


In [17]:
# checking 
tickers

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
497,XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components,"White Plains, New York",2011-11-01,1524472,2011
498,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
499,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
500,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927


In [18]:
# checking 
tickers.groupby('GICS Sector').count()

,Symbol,Security,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
GICS Sector,,,,,,,
Communication Services,22,22,22,22,22,22,22
Consumer Discretionary,50,50,50,50,50,50,50
Consumer Staples,38,38,38,38,38,38,38
Energy,22,22,22,22,22,22,22
Financials,72,72,72,72,72,72,72
Health Care,61,61,61,61,61,61,61
Industrials,78,78,78,78,78,78,78
Information Technology,69,69,69,69,69,69,69
Materials,28,28,28,28,28,28,28


In [19]:
# build 6 month time periods we will use in our analysis
years = [str(i) for i in range(2009,2025)]
intervals = []
for year in years:
    intervals.append(year+"-01-01")
    intervals.append(year+"-06-01")
intervals.append("2024-12-24")
#print(intervals)

In [20]:
# compute alpha and beta for all stocks and time periods 
stocks = list(tickers['Symbol'])

data = []

for stock in stocks:
    #print(stock, interval)
    for i in range(len(intervals)-1):
        worked, alpha,beta = alphabeta(stock, intervals[i], intervals[i+1])
        if worked:
            results = (stock, intervals[i], intervals[i+1], alpha, beta)
            data.append(results)


In [21]:
# checking 
print(len(data))

15081


In [23]:
# create a dataframe with the data 
df = pd.DataFrame(data, columns = ['Ticker', 'Start', 'End', 'Alpha', 'Beta'])

In [24]:
df

,Ticker,Start,End,Alpha,Beta
0,MMM,2009-01-01,2009-06-01,0.010121,0.940389
1,MMM,2009-06-01,2010-01-01,0.237615,0.963574
2,MMM,2010-01-01,2010-06-01,-0.001571,0.839192
3,MMM,2010-06-01,2011-01-01,-0.042458,0.926140
4,MMM,2011-01-01,2011-06-01,0.031243,1.041658
...,...,...,...,...,...
15076,ZTS,2022-06-01,2023-01-01,-0.052728,1.026287
15077,ZTS,2023-01-01,2023-06-01,0.036815,0.907690
15078,ZTS,2023-06-01,2024-01-01,0.055007,1.121700
15079,ZTS,2024-01-01,2024-06-01,-0.256295,1.175230


In [25]:
df[df['Ticker']=='NVDA']

,Ticker,Start,End,Alpha,Beta
10398,NVDA,2009-01-01,2009-06-01,0.241298,1.286515
10399,NVDA,2009-06-01,2010-01-01,0.419349,1.580953
10400,NVDA,2010-01-01,2010-06-01,-0.234234,1.541083
10401,NVDA,2010-06-01,2011-01-01,-0.006966,1.326579
10402,NVDA,2011-01-01,2011-06-01,0.140567,2.093188
10403,NVDA,2011-06-01,2012-01-01,-0.210723,1.500975
10404,NVDA,2012-01-01,2012-06-01,-0.147491,1.237695
10405,NVDA,2012-06-01,2013-01-01,-0.128127,1.366776
10406,NVDA,2013-01-01,2013-06-01,0.018754,1.144395
10407,NVDA,2013-06-01,2014-01-01,0.019700,0.792651


In [26]:
# save to a CSV file to be used for later analysis
df.to_csv('stock_dataset.csv', index = False)